<a href="https://colab.research.google.com/github/pmsk98/Graduation/blob/main/news_sentiment_classification_updown_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pwd

In [ ]:
##현대차

import pandas as pd
import os
!pip install konlpy
# 불용어 로드
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])


os.chdir('/content/gdrive/My Drive/뉴스토픽분류/')
os.getcwd()

train =pd.read_csv('hyundai_community_train_set.csv')
test=pd.read_csv('hyundai_community_test_set.csv')



train['updown_label'] = None
test['updown_label'] = None

for i in range(len(train)):
    if train['Change'][i] > 0 :
        train['updown_label'][i] = 1
    else:
        train['updown_label'][i]= 0
    

for i in range(len(test)):
    if test['Change'][i] >0:
        test['updown_label'][i] =1
    else:
        test['updown_label'][i] =0
        
for i in range(len(train)):
    train['text'][i] = str(train['title'][i])
    
for i in range(len(test)):
    test['text'][i] = str(test['title'][i])
    


import konlpy

from konlpy.tag import Okt
okt = Okt()
X_train = [] 
for sentence in train['text']: 
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_train.append(temp_X) 

X_test = [] 
for sentence in test['text']:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords]
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_test.append(temp_X)
    
    
import numpy as np 
y_train = [] 
y_test = [] 

for i in range(len(train['updown_label'])): 
    if train['updown_label'].iloc[i] == 1: 
        y_train.append(1) 
    elif train['updown_label'].iloc[i] ==0:
        y_train.append(0) 


for i in range(len(test['updown_label'])): 
    if test['updown_label'].iloc[i] == 1: 
        y_test.append(1) 
    elif test['updown_label'].iloc[i] == 0: 
        y_test.append(0) 


y_train = np.array(y_train) 
y_test = np.array(y_test)

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

from keras.preprocessing.text import Tokenizer 
max_words = 35000

MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.text)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)



from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(train.text),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test.text),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

from keras.layers import Embedding, Dense, LSTM 
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences 


model = Sequential() 
model.add(Embedding(max_words, 100)) 
model.add(LSTM(128)) 
model.add(Dense(1, activation='sigmoid')) 

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor='val_accuracy', patience=50)

model.summary()


%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
    history = model.fit(x_train, y_train,
                        epochs=100, batch_size=10, validation_split=0.1,
                        callbacks=[earlystopping])
    
#def decode_sentiment(score):
#    return 1 if score>0.5 else 0
def decode_sentiment(score):
    if score > 0.75 :
        return 1
    else:
        return 0
scores = model.predict(x_test, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

test['predict_lstm'] = None

for i in range(len(test)):
    test['predict_lstm'][i] = y_pred_1d[i]
    
    
test['predict_lstm'].value_counts()

label = test['label']
pred = test['predict_lstm']

stock_price =test[['date','Open','High','Low','Close','Volume','predict_lstm']]

stock_price=stock_price.drop_duplicates(['date'],keep='first')

stock_price = stock_price.reset_index(drop=True)

stock_price.to_csv('hyundai_community_test_2020_lstm_updown.csv')

In [ ]:
##sk
train =pd.read_csv('sk_community_train_set.csv')
test=pd.read_csv('sk_community_test_set.csv')



train['updown_label'] = None
test['updown_label'] = None

for i in range(len(train)):
    if train['Change'][i] > 0 :
        train['updown_label'][i] = 1
    else:
        train['updown_label'][i]= 0
    

for i in range(len(test)):
    if test['Change'][i] >0:
        test['updown_label'][i] =1
    else:
        test['updown_label'][i] =0
        
for i in range(len(train)):
    train['text'][i] = str(train['title'][i])
    
for i in range(len(test)):
    test['text'][i] = str(test['title'][i])
    


import konlpy

from konlpy.tag import Okt
okt = Okt()
X_train = [] 
for sentence in train['text']: 
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_train.append(temp_X) 

X_test = [] 
for sentence in test['text']:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords]
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_test.append(temp_X)
    
    
import numpy as np 
y_train = [] 
y_test = [] 

for i in range(len(train['updown_label'])): 
    if train['updown_label'].iloc[i] == 1: 
        y_train.append(1) 
    elif train['updown_label'].iloc[i] ==0:
        y_train.append(0) 


for i in range(len(test['updown_label'])): 
    if test['updown_label'].iloc[i] == 1: 
        y_test.append(1) 
    elif test['updown_label'].iloc[i] == 0: 
        y_test.append(0) 


y_train = np.array(y_train) 
y_test = np.array(y_test)

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

from keras.preprocessing.text import Tokenizer 
max_words = 35000

MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.text)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)



from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(train.text),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test.text),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

from keras.layers import Embedding, Dense, LSTM 
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences 


model = Sequential() 
model.add(Embedding(max_words, 100)) 
model.add(LSTM(128)) 
model.add(Dense(1, activation='sigmoid')) 

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor='val_accuracy', patience=50)

model.summary()


%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
    history = model.fit(x_train, y_train,
                        epochs=100, batch_size=10, validation_split=0.1,
                        callbacks=[earlystopping])
    
#def decode_sentiment(score):
#    return 1 if score>0.5 else 0
def decode_sentiment(score):
    if score > 0.75 :
        return 1
    else:
        return 0
scores = model.predict(x_test, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

test['predict_lstm'] = None

for i in range(len(test)):
    test['predict_lstm'][i] = y_pred_1d[i]
    
    
test['predict_lstm'].value_counts()

label = test['label']
pred = test['predict_lstm']

stock_price =test[['date','Open','High','Low','Close','Volume','predict_lstm']]

stock_price=stock_price.drop_duplicates(['date'],keep='first')

stock_price = stock_price.reset_index(drop=True)

stock_price.to_csv('sk_community_test_2020_lstm_updown.csv')

In [ ]:
##celltrion

train =pd.read_csv('celltrion_community_train_set.csv')
test=pd.read_csv('celltrion_community_test_set.csv')



train['updown_label'] = None
test['updown_label'] = None

for i in range(len(train)):
    if train['Change'][i] > 0 :
        train['updown_label'][i] = 1
    else:
        train['updown_label'][i]= 0
    

for i in range(len(test)):
    if test['Change'][i] >0:
        test['updown_label'][i] =1
    else:
        test['updown_label'][i] =0
        
for i in range(len(train)):
    train['text'][i] = str(train['title'][i])
    
for i in range(len(test)):
    test['text'][i] = str(test['title'][i])
    


import konlpy

from konlpy.tag import Okt
okt = Okt()
X_train = [] 
for sentence in train['text']: 
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_train.append(temp_X) 

X_test = [] 
for sentence in test['text']:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords]
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_test.append(temp_X)
    
    
import numpy as np 
y_train = [] 
y_test = [] 

for i in range(len(train['updown_label'])): 
    if train['updown_label'].iloc[i] == 1: 
        y_train.append(1) 
    elif train['updown_label'].iloc[i] ==0:
        y_train.append(0) 


for i in range(len(test['updown_label'])): 
    if test['updown_label'].iloc[i] == 1: 
        y_test.append(1) 
    elif test['updown_label'].iloc[i] == 0: 
        y_test.append(0) 


y_train = np.array(y_train) 
y_test = np.array(y_test)

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

from keras.preprocessing.text import Tokenizer 
max_words = 35000

MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.text)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)



from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(train.text),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test.text),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

from keras.layers import Embedding, Dense, LSTM 
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences 


model = Sequential() 
model.add(Embedding(max_words, 100)) 
model.add(LSTM(128)) 
model.add(Dense(1, activation='sigmoid')) 

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor='val_accuracy', patience=50)

model.summary()


%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
    history = model.fit(x_train, y_train,
                        epochs=100, batch_size=10, validation_split=0.1,
                        callbacks=[earlystopping])
    
#def decode_sentiment(score):
#    return 1 if score>0.5 else 0
def decode_sentiment(score):
    if score > 0.75 :
        return 1
    else:
        return 0
scores = model.predict(x_test, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

test['predict_lstm'] = None

for i in range(len(test)):
    test['predict_lstm'][i] = y_pred_1d[i]
    
    
test['predict_lstm'].value_counts()

label = test['label']
pred = test['predict_lstm']

stock_price =test[['date','Open','High','Low','Close','Volume','predict_lstm']]

stock_price=stock_price.drop_duplicates(['date'],keep='first')

stock_price = stock_price.reset_index(drop=True)

stock_price.to_csv('celltrion_community_test_2020_lstm_updown.csv')


In [ ]:
##samsung

train =pd.read_csv('samsung_community_train_set.csv')
test=pd.read_csv('samsung_community_test_set.csv')



train['updown_label'] = None
test['updown_label'] = None

for i in range(len(train)):
    if train['Change'][i] > 0 :
        train['updown_label'][i] = 1
    else:
        train['updown_label'][i]= 0
    

for i in range(len(test)):
    if test['Change'][i] >0:
        test['updown_label'][i] =1
    else:
        test['updown_label'][i] =0
        
for i in range(len(train)):
    train['text'][i] = str(train['title'][i])
    
for i in range(len(test)):
    test['text'][i] = str(test['title'][i])
    


import konlpy

from konlpy.tag import Okt
okt = Okt()
X_train = [] 
for sentence in train['text']: 
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_train.append(temp_X) 

X_test = [] 
for sentence in test['text']:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords]
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_test.append(temp_X)
    
    
import numpy as np 
y_train = [] 
y_test = [] 

for i in range(len(train['updown_label'])): 
    if train['updown_label'].iloc[i] == 1: 
        y_train.append(1) 
    elif train['updown_label'].iloc[i] ==0:
        y_train.append(0) 


for i in range(len(test['updown_label'])): 
    if test['updown_label'].iloc[i] == 1: 
        y_test.append(1) 
    elif test['updown_label'].iloc[i] == 0: 
        y_test.append(0) 


y_train = np.array(y_train) 
y_test = np.array(y_test)

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

from keras.preprocessing.text import Tokenizer 
max_words = 35000

MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.text)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)



from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(train.text),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test.text),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

from keras.layers import Embedding, Dense, LSTM 
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences 


model = Sequential() 
model.add(Embedding(max_words, 100)) 
model.add(LSTM(128)) 
model.add(Dense(1, activation='sigmoid')) 

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor='val_accuracy', patience=50)

model.summary()


%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
    history = model.fit(x_train, y_train,
                        epochs=100, batch_size=10, validation_split=0.1,
                        callbacks=[earlystopping])
    
#def decode_sentiment(score):
#    return 1 if score>0.5 else 0
def decode_sentiment(score):
    if score > 0.75 :
        return 1
    else:
        return 0
scores = model.predict(x_test, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

test['predict_lstm'] = None

for i in range(len(test)):
    test['predict_lstm'][i] = y_pred_1d[i]
    
    
test['predict_lstm'].value_counts()

label = test['label']
pred = test['predict_lstm']

stock_price =test[['date','Open','High','Low','Close','Volume','predict_lstm']]

stock_price=stock_price.drop_duplicates(['date'],keep='first')

stock_price = stock_price.reset_index(drop=True)

stock_price.to_csv('samsung_community_test_2020_lstm_updown.csv')

In [ ]:
##lg

train =pd.read_csv('lg_community_train_set.csv')
test=pd.read_csv('lg_community_test_set.csv')


train['updown_label'] = None
test['updown_label'] = None

for i in range(len(train)):
    if train['Change'][i] > 0 :
        train['updown_label'][i] = 1
    else:
        train['updown_label'][i]= 0
    

for i in range(len(test)):
    if test['Change'][i] >0:
        test['updown_label'][i] =1
    else:
        test['updown_label'][i] =0
        
for i in range(len(train)):
    train['text'][i] = str(train['title'][i])
    
for i in range(len(test)):
    test['text'][i] = str(test['title'][i])
    


import konlpy

from konlpy.tag import Okt
okt = Okt()
X_train = [] 
for sentence in train['text']: 
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_train.append(temp_X) 

X_test = [] 
for sentence in test['text']:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords]
    temp_X = [word for word in temp_X if len(word) > 1] # 불용어 제거 
    X_test.append(temp_X)
    
    
import numpy as np 
y_train = [] 
y_test = [] 

for i in range(len(train['updown_label'])): 
    if train['updown_label'].iloc[i] == 1: 
        y_train.append(1) 
    elif train['updown_label'].iloc[i] ==0:
        y_train.append(0) 


for i in range(len(test['updown_label'])): 
    if test['updown_label'].iloc[i] == 1: 
        y_test.append(1) 
    elif test['updown_label'].iloc[i] == 0: 
        y_test.append(0) 


y_train = np.array(y_train) 
y_test = np.array(y_test)

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

from keras.preprocessing.text import Tokenizer 
max_words = 35000

MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.text)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)



from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(train.text),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test.text),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

from keras.layers import Embedding, Dense, LSTM 
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences 


model = Sequential() 
model.add(Embedding(max_words, 100)) 
model.add(LSTM(128)) 
model.add(Dense(1, activation='sigmoid')) 

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor='val_accuracy', patience=50)

model.summary()


%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
    history = model.fit(x_train, y_train,
                        epochs=100, batch_size=10, validation_split=0.1,
                        callbacks=[earlystopping])
    
#def decode_sentiment(score):
#    return 1 if score>0.5 else 0
def decode_sentiment(score):
    if score > 0.75 :
        return 1
    else:
        return 0
scores = model.predict(x_test, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

test['predict_lstm'] = None

for i in range(len(test)):
    test['predict_lstm'][i] = y_pred_1d[i]
    
    
test['predict_lstm'].value_counts()

label = test['label']
pred = test['predict_lstm']

stock_price =test[['date','Open','High','Low','Close','Volume','predict_lstm']]

stock_price=stock_price.drop_duplicates(['date'],keep='first')

stock_price = stock_price.reset_index(drop=True)

stock_price.to_csv('lg_community_test_2020_lstm_updown.csv')